In [1]:
# pip install requests bs4 pandas firebase-admin
# !pip install pytwits # Needs token, uses API, limited accesses, can't access old stuff in order

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import csv
import json
import time

Initialize Firestore DB

In [4]:
PATH_TO_FIRESTORE_KEY = "" # Generate a JSON key from the firestore website
FILE_FOR_STORING_POSTS = "" # Make a CSV file for this

In [5]:
#load in firestore credentials
# https://firebase.google.com/docs/firestore/quickstart
# DON"T FORGET TO DELETE THE PATH WHEN PUSHING TO GITHUB!!!!
cred = credentials.Certificate(PATH_TO_FIRESTORE_KEY)
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [6]:
# Just testing this out
collection = db.collection('twit')
document = collection.document('startIndex')
print(document.get().to_dict())

{'index': 0, 'testIndex': 10}


This is what I would be storing for each twit

In [19]:
# twit_url = "https://stocktwits.com/user/message/497084294"
# print('url: ', twit_url)
# page = requests.get(twit_url)
# soup = BeautifulSoup(page.content, 'html.parser')
# twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"]) # This still works work
# print('text: ', twit_text)
# twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','') # This no longer works
# print('user: ', twit_user)

In [20]:
twit_url = "https://stocktwits.com/user/message/802"
page = requests.get(twit_url)
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.head.find("title").contents[0])

# post_contents = soup.body.find("script", attrs={"id" : "__NEXT_DATA__"})
# json_contents = json.loads(post_contents.contents[0])
# post_body = json_contents['props']['pageProps']['initialData']['message']['body']
# print(post_body)
# post_user = json_contents['props']['pageProps']['initialData']['message']['user']['username']
# print(post_user)

# Do we need to store twit URL and user URL? Those can be generated, and add much space
# Is this good?

This message does not exist | Stocktwits


In [10]:
#Testing writing to CSV file
with open(FILE_FOR_STORING_POSTS, 'a') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([10001, "greenskeptic", "$JCI only stock in my portfolio showing green today, and then only +.31. Will try to forget this day ever happened."])

In [28]:
# Crawls 1000 messages, takes a bit over 5 minutes on Google Colab, puts messages in a vector
# May or may not fail

messages_to_crawl = 1000
messages = [None] * messages_to_crawl
users = [None] * messages_to_crawl
message_numbers = [None] * messages_to_crawl

post_num = 0
crawled_num = 0
start = time.time()
while (crawled_num < messages_to_crawl):
    post_num += 1
    post_url = "https://stocktwits.com/user/message/" + str(post_num)
    page = requests.get(post_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    if soup.head.find("title").contents[0] == "This message does not exist | Stocktwits":
        continue
    else:
        post_contents = soup.body.find("script", attrs={"id" : "__NEXT_DATA__"})
        json_contents = json.loads(post_contents.contents[0])
        post_body = json_contents['props']['pageProps']['initialData']['message']['body']
        messages[crawled_num] = post_body
        post_user = json_contents['props']['pageProps']['initialData']['message']['user']['username']
        users[crawled_num] = post_user
        message_numbers[crawled_num] = post_num
        crawled_num += 1

        if crawled_num % 50 == 0:
            print(post_num, crawled_num)
end = time.time()

print(end - start)

print(messages[999])
print(users[999])

92 50
171 100
244 150
299 200
370 250
459 300
541 350
623 400
698 450
800 500
891 550
990 600
1067 650
1151 700
1230 750
1307 800
1411 850
1489 900
1592 950
1679 1000
314.3927114009857


In [17]:
# Crawls 1000 messages, appends them to the CSV file whose location you specified 
# May or may not fail
messages_to_crawl = 1000
with open(FILE_FOR_STORING_POSTS, 'a') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "post_author", "post_contents"])

    post_num = 0
    crawled_num = 0
    start = time.time()
    while (crawled_num < messages_to_crawl):
        post_num += 1
        post_url = "https://stocktwits.com/user/message/" + str(post_num)
        page = requests.get(post_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        if soup.head.find("title").contents[0] == "This message does not exist | Stocktwits":
            continue
        else:
            post_contents = soup.body.find("script", attrs={"id" : "__NEXT_DATA__"})
            json_contents = json.loads(post_contents.contents[0])
            post_contents = json_contents['props']['pageProps']['initialData']['message']['body']

            post_author = json_contents['props']['pageProps']['initialData']['message']['user']['username']

            writer.writerow([post_num, post_author, post_contents])

            crawled_num += 1

            if crawled_num % 50 == 0:
                print(post_num, crawled_num)
    end = time.time()

    print(end - start)

891 50
990 100
37.56579279899597


Stuff below was made by William

Example on how to get data - get our index flag for last message scraped

In [ ]:
# we would first start by retrieving the last good message id
doc_ref = db.collection(u'twit').document(u'startIndex')
# https://firebase.google.com/docs/firestore/query-data/get-data
doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'testIndex': 1, 'index': 0}


Example on how to push data - updating our index flag

In [ ]:
data = {
    u'testIndex': 10,
}
# https://firebase.google.com/docs/firestore/manage-data/add-data
# Updating our doc_ref from above to whatever we specify in data.index
doc_ref.set(data, merge=True)
# Check if it worked by replaying above get data example and see if it changed

update_time {
  seconds: 1668814840
  nanos: 26139000
}

Next steps for creating the actual scraper:
1. Loop through starting at index
2. https://stocktwits.com/message/# -> # should be the index
3. Use bs4 and add current data to csv as new row
4. End loop when no more new messages ~ will need to talk with team on how to determine this
5. Update index to what first bad index was
6. Push csv to storage - I will add this code in soon
7. End

In [ ]:
#this is example
headers = ['id', 'twit_url', 'twit_text', 'twit_user', 'twit_user_url']
data = []
#change range for more samples if needed
for i in range(250):
    twit_url = "https://stocktwits.com/message/" + str(i)
    page = requests.get(twit_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    twit_text = str(soup.head.find("meta", attrs={"name" : "description"})["content"]).replace(',','')
    twit_user = str(soup.head.find("meta", attrs={"name" : "twitter:title"})["content"]).replace(' | Stocktwits','')
    twit_user_url = 'https://stocktwits.com/'+twit_user.replace('@','')
    if twit_user != '@':
        data.append([i, twit_url, twit_text, twit_user, twit_user_url])
#change name if wanting multiple example csvs
with open('../csv/example.csv', 'w+') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(data)